<a href="https://colab.research.google.com/github/ddPn08/stable-diffusion-webui-colab/blob/main/automatic1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI Colab by [ddPn08](https://github.com/ddpn08/)

This colab runs from the repo [`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui)  
このColabは[`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui) を使用しています。


## 1 - Setup

### 1.1 Download repo and install

Clone git repo and setup miniconda
> Gitリポジトリをクローン、minicondaのセットアップ

In [ ]:
# @markdown ## Clone the automatic1111/stable-diffusion-webui

webui_branch = "master"  # @param {type: "string"}

! git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui
! git checkout {webui_branch}

import sys

! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append("/usr/local/lib/python3.8/site-packages/")
! rm Miniconda3-latest-Linux-x86_64.sh
! conda env update -n base -f environment-wsl2.yaml

! pip install -r requirements.txt
! mkdir repositories
! git clone https://github.com/CompVis/stable-diffusion.git repositories/stable-diffusion
! git clone https://github.com/CompVis/taming-transformers.git repositories/taming-transformers
! git clone https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
! git clone https://github.com/salesforce/BLIP.git repositories/BLIP
! pip install -r repositories/CodeFormer/requirements.txt

### 1.2 Setup models

In [ ]:
# @markdown # Load the stable-diffusion model
# @markdown > ### stable-diffusionのモデルをロード

# @markdown **Model Path Variables**
# ask for the link
print("Local Path Variables:\n")

model_filename = "sd-v1-4.ckpt"  # @param {type:"string"}
models_path = "/content/models"  # @param {type:"string"}
output_path = "/content/output"  # @param {type:"string"}
config_path = "/content/config" # @param {type:"string"}

# @markdown **Download the model if it isn't already in the `models_path` folder (Optional)**
# @markdown > モデルが`models_path`フォルダーにない場合に、モデルをダウンロードする

# @markdown To download the model, you need to have accepted the terms [HERE](https://huggingface.co/CompVis/stable-diffusion-v1-4)
# @markdown and have copied a token from [HERE](https://huggingface.co/settings/tokens)

# @markdown > モデルをダウンロードするには, [このページ](https://huggingface.co/CompVis/stable-diffusion-v1-4)で条件に同意する必要があります。
# @markdown > また、[このページ](https://huggingface.co/settings/tokens)からトークンを取得する必要があります。
download_if_missing = False  # @param {type:"boolean"}
model_url = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt"  # @param {type:"string"}
token = ""  # @param {type:"string"}

# @markdown **Google Drive Path Variables (Optional)**
mount_google_drive = True  # @param {type:"boolean"}
force_remount = False

%cd /content/
import os

mount_success = True
if mount_google_drive:
    from google.colab import drive

    try:
        drive_path = "/content/drive"
        drive.mount(drive_path, force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"  # @param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/outputs"  # @param {type:"string"}
        config_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/config" # @param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
        config_path = config_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")
        mount_success = False

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)

if not os.path.exists(f'{config_path}/config.json'):
    with open(f'{config_path}/config.json', mode='w') as f:
        f.write('{"outdir_samples": "' + output_path + '"}')

if download_if_missing:
    if not mount_success:
        print("Downloading model to " + models_path + " due to gdrive mount error")
    elif not os.path.exists(models_path + "/" + model_filename):
        ! mkdir sd-model
        %cd /content/sd-model/
        ! curl -LJ {model_url} -H {'"Authorization: Bearer ' + token + '"' if token else ""} -o {model_filename}
        ! mv /content/sd-model/{model_filename} {models_path}/
        %cd /content/
    else:
        print("Model already downloaded, moving to next step")

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")
print(f"config_path: {config_path}")

In [ ]:
# @markdown ## Download GFPGAN model
# @markdown **GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second
# @markdown > **GFPGAN** オプションで歪んだ顔を自動的に修正し、0.5秒未満で修正します

%cd stable-diffusion-webui
! curl -LOJ https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth

## 2 Optional - Set webUI settings and configs before running

In [ ]:
# @markdown # Launch preferences - Advanced
# @markdown # 詳細設定

no_half = False  # @param {type:"boolean"}
# @markdown * Do not switch the model to 16-bit floats
# @markdown > モデルを16ビットfloatに切り替えない
no_progressbar_hiding = False  # @param {type:"boolean"}
# @markdown * Do not hide progressbar in gradio UI (we hide it because it slows down ML if you have hardware acceleration in browser)
# @markdown > プログレスバーを非表示にしない (ブラウザにハードウェアアクセラレーションが場合MLが遅くなるため非表示にすることを推奨します)
medvram = False  # @param {type:"boolean"}
# @markdown * Enable stable diffusion model optimizations for sacrificing a little speed for low VRM usage
# @markdown > Stable-Diffusionモデルの最適化により、速度を少し犠牲にしてVRMの使用量を少なくすることが可能です
lowvram = False # @param {type:"boolean"}
# @markdown * Enable stable diffusion model optimizations for sacrificing a lot of speed for very low VRM usage
# @markdown > Stable-Diffusionモデルの最適化により、多くの速度を犠牲にして非常に低いVRM使用率を実現することが可能
disable_opt_split_attention = False # @param {type:"boolean"}
# @markdown * Force-disables cross-attention layer optimization
# @markdown > クロスアテンションレイヤーの最適化を強制的に無効にします

run_string_with_variables = {
    "--no-half": f"{no_half}",
    "--no-progressbar-hiding": f"{no_progressbar_hiding}",
    "--medvram": f"{medvram}",
    "--lowvram": f"{lowvram}",
    "--disable-opt-split-attention": f"{disable_opt_split_attention}"
}

only_true_vars = {k for (k, v) in run_string_with_variables.items() if v == "True"}
vars = " ".join(only_true_vars)

## 3 - Launch WebUI for stable diffusion

In [ ]:
#@markdown keep in mind that this script is set to run for ever.
#@markdown > ※注意 このスクリプトは永久に実行されます。

#@markdown # Important - click the public URL to launch WebUI in another tab
#@markdown > ### 重要 - 公開URLをクリックしてWebUIを起動してください

#@markdown ![](https://user-images.githubusercontent.com/71378929/189563599-6df78bcf-133b-41e8-a55d-8ca3783cd933.png)

! git pull
! python webui.py \
  --share \
  --ckpt-dir {models_path} \
  --ckpt {models_path}/{model_filename} \
  --ui-config-file {config_path}/ui-config.json \
  --ui-settings-file {config_path}/config.json \
  --styles-file {config_path}/styles.csv \
  $vars